# Desafio Técnico Etapa 1 - Guilherme Campos

## Objetivos
 - Escolher uma API para extração de dados
 - Formatar os dados da API de forma tabular utilizando Pandas
 - Criar gráficos, análises, tabelas, sobre os dados extraídos, utilizando bibliotecas python

### API escolhida

Após realizar a leitura das APIs disponíveis, decidi optar pela API da Food and Drug Administration (FDA), pois tenho experiência prévia em projetos de análise de dados e aprendizado de máquina com dados de saúde. Essa experiência me levou a escolher essa API, que se alinha com meu conhecimento. Após tomar a decisão de utilizar a API da FDA dediquei um tempo para ler o overview e a documentação, além disso li alguns exemplos disponíveis e realizei alguns testes de chamdas da API pelo navegador. A consulta é estruturada com a URL base da API e parâmetros específicos, como a busca, número de registros e limite, essa consulta tem a forma: 
- https://api.fda.gov/drug/label.json(endpoint base)?search=campo:termo&limit=5(limite).

### Categorias escolhidas

Escolhi essas categorias pois achei seu contexto relevante na análise de dados de sáude

- What adverse reactions are frequently reported?: Essa categoria me permite explorar as condições adversas mais frequentemente registradas, o que pode fornecer insights sobre quais medicamentos ou substâncias estão associadas a efeitos colaterais comuns

- Who reports adverse events?: Com essa informação é possível identificar quais fontes estão mais frequentimente relatando os eventos adversos, permitindo uma possível correlação 

- Adverse drug event reports by Country: Essa categoria permite a análise geográfica nos medicamentos 

- Populated with Patient Age Group code.: Analisando os dados agrupados por faixa etária, é possível entender como as reações adversas variam entre diferentes grupos de pacientes, ajudando na personalização de tratamentos e abordagens médicas.

- Outcome of the reaction at the time of last observation.: Essa categoria permite avaliar a gravidade das reações adversas, observando os desfechos e os efeitos das reações no paciente ao longo do tempo.


- The drug’s dosage form.: Essa informação pode ser útil para correlacionar a forma de administração com o tipo de reação adversa

In [46]:
#Importação das bibliotecas necessárias para a extração da API
import requests
import pandas as pd

In [42]:
#URLs escolhidas para análise: Food and Drug Administration (FDA)
urls = {
    "frequent_adverse_reactions": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=patient.drug.activesubstance.activesubstancename.exact',
    "adverse_event_reporters": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=primarysource.qualification',
    "reports_by_country": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=primarysource.reportercountry.exact',
    "patient_age_groups": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=patient.patientagegroup',
    "reaction_outcomes": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=patient.reaction.reactionoutcome',
    "drug_dosage_forms": 'https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20241219]&count=patient.drug.drugdosageform.exact',
}

#Função para extração dos dados da API que realiza uma requisição GET e retorna o JSON, inclui também tratamento de erros
def extracao(url): 
    try: 
        requisicao = requests.get(url, timeout=60)
        requisicao.raise_for_status()
        return requisicao.json()
    except requests.exceptions.RequestException as erro: 
        print(f"Erro ao acessar {url}", {erro})

# Função que itera sobre o dicionário de urls passando como parâmetro para a função extração e retorna a coluna de results da API como um dataframe pandas
def dataframe():
    dados = {}
    for nome, url in urls.items():
        try: 
            arquivo_json = extracao(url)
            if 'results' in arquivo_json and arquivo_json['results']: 
                df = pd.DataFrame(arquivo_json['results'])
                dados[nome] = df
            else: 
                print(f"Erro para a URL: {url}")
        except Exception as erro: 
            print(f"Erro para a url {url} {erro}")
            
    print("Requisições sem erro")
    return dados

#Gerando DataFrames dos dados da API e os armazenando em um dicionário
dados_api = dataframe()

#Itera sbre o dicionário de dataframe e cria uma variável global para acesso 
for nome, df in dados_api.items():
    if df is not None:
        globals()[f"df_{nome}"] = df
        print(f"Variável df_{nome} criada.")
    else: 
        print(f"Erro ao criar o DatFrame {nome}.")



Requisições sem erro
Variável df_frequent_adverse_reactions criada.
Variável df_adverse_event_reporters criada.
Variável df_reports_by_country criada.
Variável df_patient_age_groups criada.
Variável df_reaction_outcomes criada.
Variável df_drug_dosage_forms criada.


Durante essa etapa do código, inicialmente fiz a leitura individual de cada URL. No entanto, percebi que o código não estava otimizado, então reescrevi de forma mais otimizada usando funções. Após isso, encontrei dificuldades ao tentar acessar cada DataFrame individualmente, o que me levou a pesquisar sobre variáveis globais e como criá-las a partir de um dicionário. Meu pensamento para essa etapa foi de criar um dicionário contendo todas as urls , em seguida, desenvolver uma função responsável pela leitura dos dados com tratamento de erros. Após isso, implementei uma função para iterar pelas URls e transformar os dados em DataFrames. Por fim, após a dificuldade de acessar cada dataframe de forma individual e global criei uma função para iterar no dicionário de dataframes e criar uma varíavel para cada dataframe. O principal motivo que me levou a adotar essa abordagem foi que, caso novas URLs sejam adicionadas, o processo se torna mais simples e eficiente

### Primeira análise nos dados 
O objetivo da próxima etapa será analisar os tipos de dados, identificar se há presença de dados ausente e, caso necessário, realizar alterações e manipulações